In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Paths to the Tiny ImageNet dataset
TRAIN_DIR = 'tiny-imagenet-200/train'
VAL_DIR = 'tiny-imagenet-200/val'

# Hyperparameters
BATCH_SIZE = 64
NUM_EPOCHS = 20
LEARNING_RATE = 0.001
NUM_CLASSES = 200
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class TinyImageNetDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform
        self.le = LabelEncoder()
        self.le.fit(labels)
        self.encoded_labels = self.le.transform(labels)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.encoded_labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

def load_training_data():
    data = []
    labels = []
    for wnid in os.listdir(TRAIN_DIR):
        wnid_dir = os.path.join(TRAIN_DIR, wnid, 'images')
        if os.path.isdir(wnid_dir):
            for img_file in os.listdir(wnid_dir):
                img_path = os.path.join(wnid_dir, img_file)
                data.append(img_path)
                labels.append(wnid)
    return data, labels

def load_validation_data():
    val_data = []
    val_labels = []
    annotations_file = os.path.join(VAL_DIR, 'val_annotations.txt')
    val_annotations = {}
    with open(annotations_file, 'r') as f:
        for line in f:
            tokens = line.strip().split('\t')
            filename, wnid = tokens[0], tokens[1]
            val_annotations[filename] = wnid
    images_dir = os.path.join(VAL_DIR, 'images')
    for img_file in os.listdir(images_dir):
        img_path = os.path.join(images_dir, img_file)
        if img_file in val_annotations:
            val_data.append(img_path)
            val_labels.append(val_annotations[img_file])
    return val_data, val_labels

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.480, 0.448, 0.398],
                         std=[0.277, 0.269, 0.282])
])

# Load training data
train_data, train_labels = load_training_data()
train_dataset = TinyImageNetDataset(train_data, train_labels, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

# Load validation data
val_data, val_labels = load_validation_data()

# Split validation data into new validation and test sets
val_data, test_data, val_labels, test_labels = train_test_split(
    val_data, val_labels, test_size=0.5, random_state=42, stratify=val_labels)

# Create validation dataset and dataloader
val_dataset = TinyImageNetDataset(val_data, val_labels, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Create test dataset and dataloader
test_dataset = TinyImageNetDataset(test_data, test_labels, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

class ViT(nn.Module):
    def __init__(self, img_size=64, patch_size1=16, patch_size2=8, num_classes=NUM_CLASSES, dim=512, depth=6, heads=8, mlp_dim=1024):
        super(ViT, self).__init__()

        # For patch size 16x16
        num_patches1 = (img_size // patch_size1) ** 2  # 16 patches
        patch_dim1 = 3 * patch_size1 * patch_size1
        self.patch_size1 = patch_size1

        # For patch size 8x8
        num_patches2 = (img_size // patch_size2) ** 2  # 64 patches
        patch_dim2 = 3 * patch_size2 * patch_size2
        self.patch_size2 = patch_size2

        # Linear projection layers
        self.to_patch_embedding1 = nn.Linear(patch_dim1, dim)
        self.to_patch_embedding2 = nn.Linear(patch_dim2, dim)

        # Positional embeddings
        self.pos_embedding1 = nn.Parameter(torch.randn(1, num_patches1 * 4 + 1, dim))  # 16 patches * 4 repetitions + 1 [CLS]
        self.pos_embedding2 = nn.Parameter(torch.randn(1, num_patches2 + 1, dim))

        # Shared [CLS] token
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))

        # Shared Transformer Encoder
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim),
            num_layers=depth
        )

        # Learnable parameters for combining attentions
        self.gamma = nn.Parameter(torch.tensor([0.5, 0.5]))

        # Classification head
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, x):
        B, C, H, W = x.shape

        # First Pass: Patches of size 16x16
        patches1 = x.unfold(2, self.patch_size1, self.patch_size1).unfold(3, self.patch_size1, self.patch_size1)
        patches1 = patches1.contiguous().view(B, C, -1, self.patch_size1, self.patch_size1)
        patches1 = patches1.permute(0, 2, 1, 3, 4)
        patches1 = patches1.reshape(B, -1, 3 * self.patch_size1 * self.patch_size1)

        # Repeat each patch 4 times
        patches1 = patches1.unsqueeze(2).repeat(1, 1, 4, 1).reshape(B, -1, 3 * self.patch_size1 * self.patch_size1)

        # Embedding and adding [CLS] token
        tokens1 = self.to_patch_embedding1(patches1)
        cls_tokens1 = self.cls_token.expand(B, -1, -1)
        tokens1 = torch.cat((cls_tokens1, tokens1), dim=1)
        tokens1 += self.pos_embedding1[:, :tokens1.size(1), :]

        # Transformer encoding
        tokens1 = tokens1.permute(1, 0, 2)
        encoded_tokens1 = self.transformer(tokens1).permute(1, 0, 2)
        attention1 = encoded_tokens1[:, 1:, :]  # Exclude [CLS] token

        # Second Pass: Patches of size 8x8 with custom order
        patches2 = x.unfold(2, self.patch_size2, self.patch_size2).unfold(3, self.patch_size2, self.patch_size2)
        patches2 = patches2.contiguous().view(B, C, -1, self.patch_size2, self.patch_size2)
        patches2 = patches2.permute(0, 2, 1, 3, 4)
        patches2 = patches2.reshape(B, -1, 3 * self.patch_size2 * self.patch_size2)

        # Reordering patches
        order = [0,1,8,9,2,3,10,11,4,5,12,13,6,7,14,15,
                 16,17,24,25,18,19,26,27,20,21,28,29,22,23,30,31,
                 32,33,40,41,34,35,42,43,36,37,44,45,38,39,46,47,
                 48,49,56,57,50,51,58,59,52,53,60,61,54,55,62,63]
        order = torch.tensor(order).to(x.device)
        patches2 = patches2[:, order, :]

        # Embedding and adding [CLS] token
        tokens2 = self.to_patch_embedding2(patches2)
        cls_tokens2 = self.cls_token.expand(B, -1, -1)
        tokens2 = torch.cat((cls_tokens2, tokens2), dim=1)
        tokens2 += self.pos_embedding2[:, :tokens2.size(1), :]

        # Transformer encoding
        tokens2 = tokens2.permute(1, 0, 2)
        encoded_tokens2 = self.transformer(tokens2).permute(1, 0, 2)
        attention2 = encoded_tokens2[:, 1:, :]  # Exclude [CLS] token

        # Combining attentions
        gamma = torch.softmax(self.gamma, dim=0)
        combined_attention = gamma[0] * attention1 + gamma[1] * attention2

        # Aggregating combined attention
        pooled = combined_attention.mean(dim=1)  # Mean pooling over sequence length

        # Classification head
        out = self.mlp_head(pooled)

        return out

def accuracy(output, target, topk=(1, 3, 5)):
    """Computes the accuracy over the k top predictions"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def train(model, device, train_loader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    top1_acc = 0.0
    top3_acc = 0.0
    top5_acc = 0.0
    total = 0

    loop = tqdm(train_loader)
    for batch_idx, (inputs, targets) in enumerate(loop):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        acc1, acc3, acc5 = accuracy(outputs, targets, topk=(1, 3, 5))
        top1_acc += acc1.item()
        top3_acc += acc3.item()
        top5_acc += acc5.item()
        total += 1

        # Update tqdm loop
        loop.set_description(f"Epoch [{epoch}/{NUM_EPOCHS}]")
        loop.set_postfix(loss=(running_loss/total), top1_acc=(top1_acc/total), top3_acc=(top3_acc/total), top5_acc=(top5_acc/total))

    return running_loss/total, top1_acc/total, top3_acc/total, top5_acc/total

def validate(model, device, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    top1_acc = 0.0
    top3_acc = 0.0
    top5_acc = 0.0
    total = 0

    with torch.no_grad():
        loop = tqdm(val_loader)
        for batch_idx, (inputs, targets) in enumerate(loop):
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            acc1, acc3, acc5 = accuracy(outputs, targets, topk=(1, 3, 5))
            top1_acc += acc1.item()
            top3_acc += acc3.item()
            top5_acc += acc5.item()
            total += 1

            # Update tqdm loop
            loop.set_description(f"Validation")
            loop.set_postfix(loss=(running_loss/total), top1_acc=(top1_acc/total), top3_acc=(top3_acc/total), top5_acc=(top5_acc/total))

    return running_loss/total, top1_acc/total, top3_acc/total, top5_acc/total

def test(model, device, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    top1_acc = 0.0
    top3_acc = 0.0
    top5_acc = 0.0
    total = 0

    with torch.no_grad():
        loop = tqdm(test_loader)
        for batch_idx, (inputs, targets) in enumerate(loop):
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            acc1, acc3, acc5 = accuracy(outputs, targets, topk=(1, 3, 5))
            top1_acc += acc1.item()
            top3_acc += acc3.item()
            top5_acc += acc5.item()
            total += 1

            # Update tqdm loop
            loop.set_description(f"Test")
            loop.set_postfix(loss=(running_loss/total), top1_acc=(top1_acc/total), top3_acc=(top3_acc/total), top5_acc=(top5_acc/total))

    avg_loss = running_loss / total
    avg_top1_acc = top1_acc / total
    avg_top3_acc = top3_acc / total
    avg_top5_acc = top5_acc / total

    print(f"Test Loss: {avg_loss:.4f}")
    print(f"Test Acc@1: {avg_top1_acc:.2f}%")
    print(f"Test Acc@3: {avg_top3_acc:.2f}%")
    print(f"Test Acc@5: {avg_top5_acc:.2f}%")

    return avg_loss, avg_top1_acc, avg_top3_acc, avg_top5_acc

model = ViT().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

train_losses = []
val_losses = []
train_top1_acc = []
val_top1_acc = []
train_top3_acc = []
val_top3_acc = []
train_top5_acc = []
val_top5_acc = []

best_acc = 0.0

for epoch in range(1, NUM_EPOCHS + 1):
    train_loss, train_acc1, train_acc3, train_acc5 = train(model, DEVICE, train_loader, criterion, optimizer, epoch)
    val_loss, val_acc1, val_acc3, val_acc5 = validate(model, DEVICE, val_loader, criterion)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_top1_acc.append(train_acc1)
    val_top1_acc.append(val_acc1)
    train_top3_acc.append(train_acc3)
    val_top3_acc.append(val_acc3)
    train_top5_acc.append(train_acc5)
    val_top5_acc.append(val_acc5)

    # Save the best model
    if val_acc1 > best_acc:
        best_acc = val_acc1
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"Best model saved with accuracy: {best_acc:.2f}%")

    print(f"Epoch {epoch}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    print(f"Train Acc@1: {train_acc1:.2f}%, Val Acc@1: {val_acc1:.2f}%")
    print(f"Train Acc@3: {train_acc3:.2f}%, Val Acc@3: {val_acc3:.2f}%")
    print(f"Train Acc@5: {train_acc5:.2f}%, Val Acc@5: {val_acc5:.2f}%")

def plot_metrics(train_metric, val_metric, metric_name):
    epochs = range(1, NUM_EPOCHS + 1)
    plt.figure()
    plt.plot(epochs, train_metric, 'b', label=f'Training {metric_name}')
    plt.plot(epochs, val_metric, 'r', label=f'Validation {metric_name}')
    plt.title(f'Training and Validation {metric_name}')
    plt.xlabel('Epochs')
    plt.ylabel(metric_name)
    plt.legend()
    plt.savefig(f'{metric_name}.png')
    plt.show()

# Plot Loss
plot_metrics(train_losses, val_losses, 'Loss')

# Plot Top-1 Accuracy
plot_metrics(train_top1_acc, val_top1_acc, 'Top-1 Accuracy')

# Plot Top-3 Accuracy
plot_metrics(train_top3_acc, val_top3_acc, 'Top-3 Accuracy')

# Plot Top-5 Accuracy
plot_metrics(train_top5_acc, val_top5_acc, 'Top-5 Accuracy')

print("Model Parameters:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name}: {param.data.shape}")

# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

# Evaluate on test set
test_loss, test_acc1, test_acc3, test_acc5 = test(model, DEVICE, test_loader, criterion)


C:\Users\m.badzohreh\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
  0%|          | 0/1563 [00:00<?, ?it/s]